In [1]:
from bs4 import BeautifulSoup
import requests
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import datetime as datetime
import json
from dateutil import relativedelta
import webbrowser
import glob
import os
import time
#us-gaap items:'http://www.xbrlsite.com/LinkedData/BrowseObjectsByType_HTML.aspx?Type=%5BConcept%5D&Submit=Submit'
#other python code: https://github.com/lukerosiak/pysec
#FASB US GAAP Taxonomy:https://www.fasb.org/cs/ContentServer?c=Page&cid=1176169699514&d=&pagename=FASB%2FPage%2FSectionPage

## Analysis

In [2]:
engine = create_engine('sqlite:///Corp_Financials_Cash.db')
###############
#Access list of tables in db
#pd.read_sql('SELECT * FROM sqlite_master',engine)
##############
df = pd.read_sql('SELECT * FROM sp_500_tables', engine)
df.drop_duplicates(inplace = True)

In [3]:
sp_500 = pd.read_sql('SELECT * FROM sp_500', engine)
sp_500.drop_duplicates(subset = ['CIK'],inplace = True)

In [4]:
df['CIK'] = df['CIK'].astype('int')

In [5]:
df = sp_500[['GICS Sector','CIK','Ticker symbol']].merge(df, on = ['CIK'], how = 'right')

In [6]:
#################
# Change CutDate dtype to datetime
#################
df['CutDate'] = df.CutDate.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d') if x != '' else x)

In [7]:
#################
# Wrangle reporting_period to convert bytesarray to integers.
#################

convert_date = {'\x00\x00\x00\x00\x00\x00\x00\x00':'0','\x03\x00\x00\x00\x00\x00\x00\x00':'3','\x06\x00\x00\x00\x00\x00\x00\x00':'6',\
               '\x0c\x00\x00\x00\x00\x00\x00\x00':'12','\x07\x00\x00\x00\x00\x00\x00\x00':'6','\t\x00\x00\x00\x00\x00\x00\x00':'9',\
               '\x04\x00\x00\x00\x00\x00\x00\x00':'3','\x01\x00\x00\x00\x00\x00\x00\x00':'1',')\x00\x00\x00\x00\x00\x00\x00':'100',\
               '\n\x00\x00\x00\x00\x00\x00\x00':'9','\x05\x00\x00\x00\x00\x00\x00\x00':'5'}
normalized = {7:6,4:3,10:9}
df['reporting_period'] = df.reporting_period.apply(lambda x: convert_date[x.decode()] if type(x) == bytes else x)
df['reporting_period'] = df.reporting_period.apply(lambda x: int(x) if x != '' else x)
df['reporting_period'] = df.reporting_period.apply(lambda x: normalized[x] if x in normalized else x)

In [8]:
def period_comparison(df, table_name_list,item, member = False, period = 'min', convert_float = True):
    ##########################
    # This function will return values comparison of the interested item across different reporting period.
    # member: True or False - False will only consider Date_Only member value
    # period: 'min' or 'max' - min or max of reporting periods, e.g. single quarter or YTD
    #convert_float: True or False - convert values to float or not
    #depth always pick the smallest value
    #contextref with the smallest length - if duplicated values still exist after all above
    #########################
    df_table = df.loc[df['table_name'].str.contains('|'.join(table_name_list)),:]
    df_table = df_table[df_table['us-gaap'] == item.lower()]
    
    df_table['contextref_len'] = df_table['contextref'].apply(lambda x: len(x))
    #groupby adds value to prevent leaving out certain items with the same member items
    #df_table = df_table.groupby(by = ['CIK','CutDate','us-gaap','reporting_period','member','value'])\
    #.agg({'contextref_len':'min','depth':'min','decimals':'max'}).reset_index().merge(df_table, on = ['CIK','CutDate','reporting_period','us-gaap','contextref_len','depth','decimals','member','value'], how = 'left')
    #
    df_table = df_table.groupby(by = ['CIK','CutDate','us-gaap','reporting_period','member'])\
    .agg({'contextref_len':'min','depth':'min','decimals':'max'}).reset_index().merge(df_table, on = ['CIK','CutDate','reporting_period','us-gaap','contextref_len','depth','decimals','member'], how = 'left')

    if member == False:
        df_table = df_table[df_table.member == 'Date_Only']
        
    #df_main = df_table.groupby(['CIK','reporting_period','member','value']).agg({'CutDate':['max','min']}).reset_index()
    #df_main.columns = ['CIK','reporting_period','member','value','this_period','last_period']
    #
    df_main = df_table.groupby(['CIK','reporting_period','member']).agg({'CutDate':['max','min']}).reset_index()
    df_main.columns = ['CIK','reporting_period','member','this_period','last_period']    
    
    if period == 'min':
        #df_main = df_main.groupby(by = ['CIK','member','value'])['reporting_period'].min().reset_index().merge(df_main, on = ['CIK','reporting_period','member'], how = 'left')
        df_main = df_main.groupby(by = ['CIK','member'])['reporting_period'].min().reset_index().merge(df_main, on = ['CIK','reporting_period','member'], how = 'left')
    if period == 'max':
        #df_main = df_main.groupby(by = ['CIK','member','value'])['reporting_period'].max().reset_index().merge(df_main, on = ['CIK','reporting_period','member'], how = 'left')
        df_main = df_main.groupby(by = ['CIK','member'])['reporting_period'].max().reset_index().merge(df_main, on = ['CIK','reporting_period','member'], how = 'left')
    
    #df_main = df_main.merge(df_table[['CIK','GICS Sector','value','reporting_period','CutDate','member']],\
    #            left_on = ['CIK','reporting_period','member','this_period','value'], right_on = ['CIK','reporting_period','member','CutDate','value'], how = 'left')\
    #.merge(df_table[['CIK','value','reporting_period','CutDate','member']], \
    #       left_on = ['CIK','reporting_period','member','last_period'], right_on = ['CIK','reporting_period','member','CutDate'], how = 'left')\
    #.rename(columns = {'value_x': 'value_this_period','value_y':'value_last_period'})\
    #.drop(['CutDate_x','CutDate_y'], axis = 1)

    df_main = df_main.merge(df_table[['CIK','GICS Sector','value','reporting_period','CutDate','member']],\
                left_on = ['CIK','reporting_period','member','this_period'], right_on = ['CIK','reporting_period','member','CutDate'], how = 'left')\
    .merge(df_table[['CIK','value','reporting_period','CutDate','member']], \
           left_on = ['CIK','reporting_period','member','last_period'], right_on = ['CIK','reporting_period','member','CutDate'], how = 'left')\
    .rename(columns = {'value_x': 'value_this_period','value_y':'value_last_period'})\
    .drop(['CutDate_x','CutDate_y'], axis = 1)
    
    
    if convert_float == True:
        df_main['value_this_period'] = df_main['value_this_period'].astype('float')
        df_main['value_last_period'] = df_main['value_last_period'].astype('float')
        df_main['value_diff'] = df_main['value_this_period'] - df_main['value_last_period']
    df_main.drop_duplicates(inplace = True)
        
    return df_main

In [65]:
#################
# Reporting Segment might not contain geoggraphy information.
#################
#df[df['us-gaap'] == 'us-gaap:OperatingIncomeLoss'.lower()]['table_name'].value_counts()
#df[df['table_name'] == '790000 - Disclosure - Segment Reporting']['us-gaap'].value_counts()
#us-gaap:operatingincomeloss
#us-gaap:revenues
#us-gaap:profitloss
#us-gaap:incomelossfromcontinuingoperationsbeforeincometaxesextraordinaryitemsnoncontrollinginterest
a = period_comparison(df,['790000'],'us-gaap:operatingincomeloss',member = True,convert_float = False)
a['member'].value_counts()

Date_Only                                             345
Operating Segments [Member]                           129
Corporate, Non-Segment [Member]                        56
us                                                     37
Segment Reconciling Items [Member]                     28
Intersegment Eliminations [Member]                     17
Other Segments [Member]                                14
Corporate and Other [Member]                           12
Corporate Segment [Member]                              8
northamericasegmentmember                               6
financialservicesmember                                 5
embeddedprocessingmember                                4
kingdigitalentertainmentmember                          4
windowsandotherspecialtyproductsmember                  4
decorativearchitecturalproductsmember                   4
cabinetryproductsmember                                 4
papermember                                             4
magnesiaspecia

## Effective Tax Rate

In [9]:
#'us-gaap:incometaxexpensebenefit'
#'us-gaap:incomelossfromcontinuingoperationsbeforeincometaxesextraordinaryitemsnoncontrollinginterest'
#'us-gaap:netincomeloss'
#us-gaap:incomelossfromcontinuingoperationsbeforeincometaxesminorityinterestandincomelossfromequitymethodinvestments
tax = period_comparison(df, ['124100'], 'us-gaap:incometaxexpensebenefit')
income = period_comparison(df, ['124100'], 'us-gaap:incomelossfromcontinuingoperationsbeforeincometaxesextraordinaryitemsnoncontrollinginterest')

income_2 = period_comparison(df, ['124100'], 'us-gaap:incomelossfromcontinuingoperationsbeforeincometaxesminorityinterestandincomelossfromequitymethodinvestments')

income = pd.concat([income,income_2], ignore_index = True).drop_duplicates(subset = ['CIK'])

In [11]:
##########
# Failed to extract income before tax items
#income before tax items in disclosure tables, instead of income statement
##########
c = tax.merge(income, on = 'CIK', how = 'outer')
c.loc[c.reporting_period_y.isnull(),'CIK']

35       30625
68       55785
76       63908
82       72333
83       72741
94       80424
193     822416
195     827052
235     920148
305    1111711
316    1137411
369    1534701
376    1585364
Name: CIK, dtype: int64

In [16]:
##########
# Check if this period and last period arethe same date
#########
tax.loc[tax['this_period'] == tax['last_period']]

,CIK,member,reporting_period,this_period,last_period,GICS Sector,value_this_period,value_last_period,value_diff
97,91142,Date_Only,0,2018-03-21,2018-03-21,Industrials,1700000.0,1700000.0,0.0


In [10]:
#################
# Effective Tax Rate
#################
eff_tax = tax.merge(income, on = ['CIK','reporting_period','this_period','last_period'], how = 'inner', suffixes = ['_tax','_income'])
eff_tax['tax_rate_this_period'] = eff_tax['value_this_period_tax']*100/eff_tax['value_this_period_income']
eff_tax['tax_rate_last_period'] = eff_tax['value_last_period_tax']*100/eff_tax['value_last_period_income']
eff_tax['tax_rate_diff'] = eff_tax['tax_rate_this_period'] - eff_tax['tax_rate_last_period']

In [11]:
##############
# Descriptive stats on over tax rate change
#Some extreme tax rates are as results of tax benefit, net loss and M&A activities
##############

#eff_tax.loc[((eff_tax['tax_rate_diff'] > 100) | (eff_tax['tax_rate_diff'] < -100))]
eff_tax.describe()[['tax_rate_this_period','tax_rate_last_period','tax_rate_diff']]

,tax_rate_this_period,tax_rate_last_period,tax_rate_diff
count,380.000000,380.000000,380.000000
mean,38.328866,32.927298,5.401568
std,301.740112,156.627729,343.741050
min,-915.789474,-442.079312,-2880.252765
25%,13.280069,18.654283,-13.801093
50%,21.085253,27.720609,-8.060193
75%,25.143070,35.021859,0.966425
max,5514.583333,2900.000000,5494.551600


In [12]:
eff_tax.groupby(by = ['GICS Sector_tax']).agg({'tax_rate_diff':['median','mean']})

tax_rate_diff           
                                  median       mean
GICS Sector_tax                                    
Consumer Discretionary         -9.977336 -13.949964
Consumer Staples               -9.576500   0.338236
Energy                         -3.367808 -93.123459
Health Care                    -3.537313  92.891947
Industrials                    -9.972581 -13.322688
Information Technology         -4.513045   3.980909
Materials                      -3.595942 -19.544136
Telecommunication Services    -12.548433 -34.843604
Utilities                      -8.830870  46.551038

## Cash & Investment

In [9]:
#############
#Extract cash items from 330000 - investment disclosure & 333000 Equity method investment
############
# 330000 table
mapping_cash = pd.ExcelFile('Mapping.xls').parse('330000')

mapped = pd.DataFrame(columns = ['CIK', 'member', 'reporting_period', 'this_period', 'last_period',\
                        'GICS Sector', 'value_this_period', 'value_last_period', 'value_diff','us-gaap'])
mapping_item = mapping_cash['us-gaap']
for i in mapping_item:
    mapped_i = period_comparison(df,['330000'],i, member = True)
    mapped_i['us-gaap'] = np.repeat(i,len(mapped_i.index))
    mapped = pd.concat([mapped,mapped_i], ignore_index = True)

# 333000 table
new_df = pd.DataFrame(columns = ['CIK'], data = list(set(mapped['CIK']))).merge(df, on = ['CIK'], how = 'inner')
mapping_cash = pd.ExcelFile('Mapping.xls').parse('333000')
mapping_item = mapping_cash['us-gaap']
for i in mapping_item:
    try:
        mapped_i = period_comparison(new_df,['333000'],i, member = True)
        mapped_i['us-gaap'] = np.repeat(i,len(mapped_i.index))
        mapped = pd.concat([mapped,mapped_i], ignore_index = True)
    except:
        #print(i)
        continue
mapped = mapped.loc[mapped['member'].str.contains('fair value', case = False, na = False) == False,:]
#mapped['value_this_period'] = mapped['value_this_period'].apply(lambda x: x/1000000)
#mapped['value_last_period'] = mapped['value_last_period'].apply(lambda x: x/1000000)
#mapped['value_diff'] = mapped['value_diff'].apply(lambda x: x/1000000)

us-gaap:equitymethodinvestmentquotedmarketvalue
us-gaap:equitymethodinvestmentsoldcarryingamount


In [10]:
##################
# Find companies left out of 330000
##################
no_investment = []
for i in list(set(df['CIK'])):
    if i not in list(set(mapped['CIK'])):
        no_investment.append(i)
c = pd.DataFrame(columns = ['CIK'], data = no_investment).merge(df, on = ['CIK'], how = 'inner')

In [11]:
##################
# Extract cash items from 815000 - Fair value disclosure for the remaining companies
##################
mapping_cash = pd.ExcelFile('Mapping.xls').parse('815000')

mapping_item = mapping_cash['us-gaap']
for i in mapping_item:
    try:
        mapped_i = period_comparison(c,['815000'],i, member = True)
        mapped_i['us-gaap'] = np.repeat(i,len(mapped_i.index))
        mapped = pd.concat([mapped,mapped_i], ignore_index = True)
    except:
        #print(i)
        continue
mapped = mapped.loc[mapped['member'].str.contains('fair value', case = False, na = False) == False,:]
mapped['value_this_period'] = mapped['value_this_period'].apply(lambda x: x/1000000)
mapped['value_last_period'] = mapped['value_last_period'].apply(lambda x: x/1000000)
mapped['value_diff'] = mapped['value_diff'].apply(lambda x: x/1000000)

us-gaap:mortgagesheldforsalefairvaluedisclosure
us-gaap:fairvalueestimatenotpracticableequitymethodinvestments
us-gaap:fairvalueestimatenotpracticableinvestments
us-gaap:mortgagebackedsecuritiesavailableforsalefairvaluedisclosure
us-gaap:assetsheldforsalelonglivedfairvaluedisclosure
us-gaap:investmentsinaffiliatessubsidiariesassociatesandjointventuresfairvaluedisclosure
us-gaap:investmentsnetassetvalue
us-gaap:alternativeinvestmentfairvaluebyfairvaluehierarchylevelandnavextensiblelist
us-gaap:fairvalueestimatenotpracticablecashandcashequivalents
us-gaap:costmethodinvestmentsfairvaluedisclosure


In [12]:
##################
# Find companies left out of both 330000 & 815000
##################
no_investment = []
for i in list(set(df['CIK'])):
    if i not in list(set(mapped['CIK'])):
        no_investment.append(i)
c = pd.DataFrame(columns = ['CIK'], data = no_investment).merge(df, on = ['CIK'], how = 'inner')

In [13]:
############
#Aggregate balance sheet cash items of the rest of the companies without investment or fair value disclosure
############


cash_item = pd.ExcelFile('Mapping.xls').parse('104000')['us-gaap']

total_cash = pd.DataFrame(columns = ['CIK', 'member', 'reporting_period', 'this_period', 'last_period',\
                        'GICS Sector', 'value_this_period', 'value_last_period', 'value_diff','us-gaap'])
for i in cash_item:
    cash = period_comparison(c,['104000'],i, member = False)
    cash['us-gaap'] = np.repeat(i, len(cash.index))
    total_cash = pd.concat([total_cash, cash], ignore_index = True)
total_cash['value_this_period'] = total_cash['value_this_period'].apply(lambda x:x/1000000)
total_cash['value_last_period'] = total_cash['value_last_period'].apply(lambda x: x/1000000)
total_cash['value_diff'] = total_cash['value_diff'].apply(lambda x: x/1000000)
#total_cash.groupby(['CIK'])['value_this_period'].sum().reset_index().sort_values(by = ['value_this_period'], ascending  = False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1457: RuntimeWarning: divide by zero encountered in longlong_scalars
  stride //= shape[i]


In [14]:
mapped = pd.concat([mapped, total_cash], ignore_index = True)

In [58]:
###########
# Companies left out of cash & investment analysis due to failue to extract cash and investment items from xbrl
##########
no_investment = []
for i in list(set(df['CIK'])):
    if i not in list(set(mapped['CIK'])):
        no_investment.append(i)
c = pd.DataFrame(columns = ['CIK'], data = no_investment).merge(df, on = ['CIK'], how = 'inner')
set(c['CIK'])

{354908, 798354, 1519751}

### Total Cash

In [15]:
strategic_invest = {'us-gaap:equitymethodinvestments':'Equity Method',\
 'us-gaap:investmentsinaffiliatessubsidiariesassociatesandjointventures':'Equity Method',\
'us-gaap:equitysecuritieswithoutreadilydeterminablefairvalueamount':'Cost Method',\
'us-gaap:costmethodinvestmentsaggregatecarryingamountnotevaluatedforimpairment':'Cost Method',\
'us-gaap:costmethodinvestments':'Cost Method',\
'us-gaap:costmethodinvestmentsfairvaluedisclosure':'Cost Method'}


In [16]:
#############
#Aggregate total investment
##320193/1018724/1652044/789019/50863/1326801/858877
############
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.apply.html
total_cash_agg = mapped.groupby(by = ['CIK','GICS Sector','us-gaap']).apply(lambda x: sum(x.value_this_period*['Date_Only' in i for i in x.member]) \
                                             if sum(['Date_Only' in i for i in x.member]) > 0\
                                                 else sum(x.value_this_period)).reset_index()

total_cash = pd.ExcelFile('Mapping.xls').parse('total_cash')
total_cash = total_cash[total_cash.inclusion == 'V']
total_cash = total_cash.merge(total_cash_agg,on = ['CIK','us-gaap'], how = 'left').rename(columns = {0:'value'})
total_cash['Cat'] = total_cash['us-gaap'].apply(lambda x: strategic_invest[x] if x in strategic_invest else 'investment')

In [27]:
total_cash.groupby(['Cat'])['value'].sum().reset_index()

,Cat,value
0,Cost Method,12160.712
1,Equity Method,157542.893
2,investment,1482870.614


In [17]:
############
#Rank of total investment
###########
agg = total_cash.groupby(by = ['CIK','GICS Sector'])['value'].sum().reset_index().sort_values(by = ['value'],ascending = False)\
.merge(sp_500[['CIK','Security']], on = ['CIK'], how = 'left')
agg.head(20)


,CIK,GICS Sector,value,Security
0,320193,Information Technology,243743.0,Apple Inc.
1,789019,Information Technology,135630.0,Microsoft Corp.
2,1652044,Information Technology,100611.0,Alphabet Inc Class A
3,1341439,Information Technology,45641.0,Oracle Corp.
4,882095,Health Care,45067.0,Gilead Sciences
5,64803,Consumer Staples,43911.0,CVS Health
6,1109357,Utilities,39990.0,Exelon Corp.
7,858877,Information Technology,38710.0,Cisco Systems
8,40545,Industrials,37205.0,General Electric
9,731766,Health Care,34216.0,United Health Group Inc.


In [19]:
agg.head(20)['value'].sum()/agg['value'].sum()

0.6132464662393485

### Investment Composition

In [51]:
############
#Check if the total cash mapping table items are current period item --> some items only apply to previous period
#First step: find all current period items in mapped table
#Second step: merge with total cash mapping table and find null values
###########
check = mapped.groupby(['CIK'])['this_period'].max().reset_index()\
.merge(mapped, on = ['CIK','this_period'], how = 'left')\
.groupby(by = ['CIK','us-gaap'])['this_period'].nunique().reset_index()\
.merge(pd.ExcelFile('Mapping.xls').parse('total_cash'), on = ['CIK','us-gaap'], how = 'outer')
check = check.loc[check['us-gaap'].str.contains('maturit') == False,:]
###########
#inclusion = V & this_period is null --> previous period values was included
#inclusion is null & this_period is not null --> current period items failed to be included.
###########
#check[(check['inclusion'].isnull()) | (check['this_period'].isnull())][check['inclusion'] == 'V'].merge(total_cash, on = ['CIK'], how = 'left')

check[(check['inclusion'].isnull()) | (check['this_period'].isnull())]
# Changed some mapping items but not all since cash amouts are already small.

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,CIK,us-gaap,this_period,inclusion,GICS Sector,value,Security
0,9389,us-gaap:restrictedcashandcashequivalentsnoncur...,NaN,V,Materials,558.000,Ball Corp
1,68505,us-gaap:availableforsalesecurities,NaN,V,Information Technology,29.000,Motorola Solutions Inc.
2,73309,us-gaap:shortterminvestments,NaN,V,Materials,2345.353,Nucor Corp.
3,77360,us-gaap:disposalgroupincludingdiscontinuedoper...,NaN,V,Industrials,105.700,Pentair plc
4,723254,us-gaap:marketablesecurities,NaN,V,Industrials,43.819,Cintas Corporation
5,811156,us-gaap:availableforsalesecurities,NaN,V,Utilities,95.000,CMS Energy
6,813828,us-gaap:disposalgroupincludingdiscontinuedoper...,NaN,V,Consumer Discretionary,261.000,CBS Corp.
7,874716,us-gaap:availableforsalesecurities,NaN,V,Health Care,284.255,IDEXX Laboratories
8,875320,us-gaap:availableforsalesecuritiesequitysecuri...,NaN,V,Health Care,1457.731,Vertex Pharmaceuticals Inc
9,1065696,us-gaap:disposalgroupincludingdiscontinuedoper...,NaN,V,Consumer Discretionary,554.971,LKQ Corporation


In [138]:
df[(df['CIK'] == 64803) & (df['member'] != 'Date_Only')]\
.loc[df['table_name'].str.contains('|'.join(['330000','815000']))]
#[df['CutDate'] == '2018-07-01']['us-gaap'].value_counts()
#2017-12-31
mapped[mapped['CIK'] == 64803]
#total_cash_agg[total_cash_agg['CIK'] == 804328]
#t[t['CIK'] == 731766]

,CIK,GICS Sector,last_period,member,reporting_period,this_period,us-gaap,value_diff,value_last_period,value_this_period
1820,64803,Consumer Staples,2017-12-31,Date_Only,0,2018-06-30,us-gaap:cashandcashequivalentsatcarryingvalue,42119.0,1696.0,43815.0
2021,64803,Consumer Staples,2017-12-31,Date_Only,0,2018-06-30,us-gaap:shortterminvestments,-15.0,111.0,96.0


In [24]:
###############
# No member breakdown for investment item: ['1341439' (Oracle used self defined item names instead of us-gaap 
#(with id number)),'40545' (changed item name),
#'1075531' (changed item name),'200406' (same current and last period),'80424' (wrong table though still 
#without detailed member info),'1326160'(changed item name)]
#'6201' (same current and last period)
###############
member_only = mapped[(mapped['member'] != 'Date_Only')].loc[mapped['this_period'] != mapped['last_period']]
##############
#import cash mapping table, merge with member only and total cash(to gauge significance)
#Focus on available for sales or marketable securties
#Question: since there are numerous missing value due to item name change, should I consider them?
#############
test = pd.ExcelFile('Mapping.xls').parse('total_cash')\
.merge(member_only.groupby(['CIK','us-gaap'])['value_this_period'].sum().reset_index(),on = ['CIK','us-gaap'], how = 'left')\
.merge(agg, on = ['CIK'], how = 'left').sort_values(by = ['value'], ascending = False)

test[test['inclusion'] == 'V'].loc[test['us-gaap'].str.contains('|'.join(['available','marketable'])),:]\
[test['value_this_period'].isnull()].loc[test['us-gaap'].str.contains('equity') == False]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,CIK,us-gaap,inclusion,value_this_period,GICS Sector,value,Security
717,1341439,us-gaap:availableforsalesecuritiescurrent,V,NaN,Information Technology,45641.000,Oracle Corp.
94,40545,us-gaap:availableforsalesecuritiesdebtsecurities,V,NaN,Industrials,37205.000,General Electric
547,1018724,us-gaap:marketablesecuritiescurrent,V,NaN,Consumer Discretionary,26743.000,Amazon.com Inc.
617,1075531,us-gaap:availableforsalesecuritiesdebtsecurities,V,NaN,Consumer Discretionary,10929.183,Booking Holdings Inc
230,200406,us-gaap:availableforsalesecurities,V,NaN,Health Care,9728.000,Johnson & Johnson
194,80424,us-gaap:availableforsalesecurities,V,NaN,Consumer Staples,9281.000,Procter & Gamble
833,1666700,us-gaap:marketablesecuritiescurrent,V,NaN,Materials,7290.000,DowDuPont
17,6201,us-gaap:availableforsalesecuritiesdebtsecuriti...,V,NaN,Industrials,4381.000,American Airlines Group
712,1326160,us-gaap:availableforsalesecuritiesdebtsecurities,V,NaN,Utilities,3440.000,Duke Energy
282,715957,us-gaap:availableforsalesecuritiesdebtsecurities,V,NaN,Utilities,3287.000,Dominion Energy


In [25]:
##########
#Significance of available member data: total cash of companies with available member data v.s. sum of total cash
# Total 105 companies (some companies have no cash investment)
#########
#mapped[mapped['us-gaap'].str.contains('maturit')].loc[mapped['this_period'] != mapped['last_period']].member.value_counts()
#mapped[(mapped['member'] != 'Date_Only')].loc[mapped['this_period'] != mapped['last_period']]
test[test['inclusion'] == 'V'].loc[test['us-gaap'].str.contains('|'.join(['available','marketable'])),:]\
[test['value_this_period'].isnull() == False].loc[test['us-gaap'].str.contains('equity') == False]\
.drop_duplicates(subset = ['CIK']).value.sum()/agg['value'].sum()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


0.6455812197321962

In [18]:
#################
#Investment Compositions
#################

member_only = mapped[(mapped['member'] != 'Date_Only')].loc[mapped['this_period'] != mapped['last_period']]
member_list = pd.ExcelFile('Mapping.xls').parse('member')
cash_item = pd.ExcelFile('Mapping.xls').parse('total_cash')
cash_item = cash_item[cash_item['inclusion'] == 'V']
##############
#import cash mapping table, merge with member only and total cash(to gauge significance)
##############
investment = cash_item\
.merge(member_only ,on = ['CIK','us-gaap'], how = 'left')\
.merge(member_list, on = ['member'], how = 'left')

investment = investment[investment['Cat'].isnull() == False]

In [30]:
################
#Check if member add up > total cash
###############

check_member_with_total_cash = investment.groupby(['CIK'])['value_this_period'].sum().reset_index().merge(agg, on = ['CIK'], how = 'left')
error = check_member_with_total_cash[check_member_with_total_cash['value_this_period'] > check_member_with_total_cash['value']]
########
#Details
########
pd.DataFrame(columns = ['CIK'],data = list(error['CIK'])).merge(investment,on = ['CIK'], how = 'left')

,CIK,us-gaap,inclusion,GICS Sector,last_period,member,reporting_period,this_period,value_diff,value_last_period,value_this_period,Cat
0,14272,us-gaap:availableforsalesecurities,V,Health Care,2017-12-31,Certificates of Deposit [Member],0,2018-06-30,55.000,141.000,196.000,Cash
1,14272,us-gaap:availableforsalesecurities,V,Health Care,2017-12-31,Commercial Paper [Member],0,2018-06-30,-50.000,50.000,0.000,CP
2,14272,us-gaap:availableforsalesecurities,V,Health Care,2017-12-31,Corporate Debt Securities [Member],0,2018-06-30,-687.000,3548.000,2861.000,Corp
3,14272,us-gaap:availableforsalesecurities,V,Health Care,2017-12-31,Equity Securities [Member],0,2018-06-30,-67.000,67.000,0.000,Equity
4,14272,us-gaap:availableforsalesecurities,V,Health Care,2017-12-31,equityandfixedincomefundsmember,0,2018-06-30,509.000,132.000,641.000,Other
5,875045,us-gaap:availableforsalesecuritiesdebtsecurities,V,Health Care,2017-12-31,Asset-backed Securities [Member],0,2018-06-30,-383.400,643.400,260.000,ABS
6,875045,us-gaap:availableforsalesecuritiesdebtsecurities,V,Health Care,2017-12-31,Corporate Debt Securities [Member],0,2018-06-30,-564.300,2609.800,2045.500,Corp
7,875045,us-gaap:availableforsalesecuritiesdebtsecurities,V,Health Care,2017-12-31,US Treasury and Government [Member],0,2018-06-30,-1090.600,1919.300,828.700,GOV
8,875045,us-gaap:availableforsalesecuritiesdebtsecurities,V,Health Care,2017-12-31,corporatedebtsecuritiescurrentmember,0,2018-06-30,444.900,1039.300,1484.200,Corp
9,875045,us-gaap:availableforsalesecuritiesdebtsecurities,V,Health Care,2017-12-31,corporatedebtsecuritiesnoncurrentmember,0,2018-06-30,-1009.200,1570.500,561.300,Corp


In [24]:
investment.groupby(['Cat']).agg({'value_this_period':'sum','value_last_period':'sum','value_diff':'sum'}).reset_index()\
.sort_values(by = ['value_diff'], ascending = False)

,Cat,value_this_period,value_last_period,value_diff
4,Cash,61140.532,53743.889,7396.643
3,CP,14381.610,8770.520,5611.090
14,MMF,42597.733,37591.735,5005.998
15,Other,58849.355,54729.850,4119.505
12,MBS,15269.454,14885.212,384.242
2,CMBS,1604.689,1610.570,-5.881
11,GOV & Corp,104.498,184.452,-79.954
1,Agency,418.155,524.698,-106.543
7,Foerign Securities,145.000,277.000,-132.000
8,Foreign Corp,3211.100,3541.900,-330.800


In [47]:
#2018/6/30 accounts for majority of this_period, while over half of the last_period was 12/31/2017.
investment.groupby(['this_period','last_period'])['CIK'].nunique().reset_index().sort_values(by = ['CIK'], ascending = False)

,this_period,last_period,CIK
17,2018-06-30,2017-12-31,69
18,2018-06-30,2018-03-31,8
15,2018-06-30,2017-09-30,8
13,2018-06-30,2017-06-30,5
20,2018-07-01,2017-12-31,4
16,2018-06-30,2017-12-30,4
26,2018-07-31,2017-10-31,3
25,2018-07-31,2017-07-31,2
5,2018-05-31,2017-11-30,2
10,2018-06-29,2017-06-30,2


## Cash Flow Item

### Purchases and Sales of Securities

In [19]:
ps = []
for i in ['PaymentsToAcquireAvailableForSaleSecuritiesDebt',\
'PaymentsToAcquireHeldToMaturitySecurities',\
'PaymentsToAcquireMarketableSecurities',\
'ProceedsFromSaleOfAvailableForSaleSecuritiesDebt',\
'ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities',\
'ProceedsFromSaleAndMaturityOfAvailableForSaleSecurities',\
'ProceedsFromSaleOfHeldToMaturitySecurities',\
'ProceedsFromMaturitiesPrepaymentsAndCallsOfHeldToMaturitySecurities',\
'ProceedsFromSaleAndMaturityOfHeldToMaturitySecurities',\
'ProceedsFromSaleAndMaturityOfMarketableSecurities',\
'ProceedsFromSaleAndMaturityOfOtherInvestments',\
'ProceedsFromSaleMaturityAndCollectionsOfInvestments',\
'PaymentsToAcquireAvailableForSaleSecuritiesDebt',\
'PaymentsToAcquireAvailableForSaleSecuritiesEquity',\
'PaymentsToAcquireAvailableForSaleSecurities',\
'PaymentsToAcquireHeldToMaturitySecurities',\
'PaymentsToAcquireMarketableSecurities',\
'ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities',\
'ProceedsFromSaleAndMaturityOfAvailableForSaleSecurities',\
'ProceedsFromSaleOfHeldToMaturitySecurities',\
'ProceedsFromMaturitiesPrepaymentsAndCallsOfHeldToMaturitySecurities',\
'ProceedsFromSaleAndMaturityOfHeldToMaturitySecurities',\
'ProceedsFromSaleAndMaturityOfMarketableSecurities',\
'ProceedsFromSaleAndMaturityOfOtherInvestments',\
'ProceedsFromSaleMaturityAndCollectionsOfInvestments',\
'ProceedsFromSaleOfAvailableForSaleSecuritiesDebt',\
'ProceedsFromSaleOfAvailableForSaleSecuritiesEquity',\
'ProceedsFromSaleOfAvailableForSaleSecurities',\
'ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities',\
'ProceedsFromSaleAndMaturityOfAvailableForSaleSecurities',\
'PaymentsToAcquireOtherInvestments',\
'PaymentsToAcquireInvestments',\
'ProceedsFromSaleOfAvailableForSaleSecurities',\
'PaymentsToAcquireTradingSecuritiesHeldforinvestment',\
'ProceedsFromSaleAndMaturityOfTradingSecuritiesHeldforinvestment']:
    ps.append('us-gaap:' + i.lower())
ps = list(set(ps))

In [20]:
df_ps = pd.DataFrame(columns = ['CIK', 'member', 'reporting_period', 'this_period', 'last_period',
       'GICS Sector', 'value_this_period', 'value_last_period', 'value_diff','us-gaap'])
for i in ps:
    try:
        df_c = period_comparison(df,['152200','330000'],i,period = 'max')
        df_c['us-gaap'] = np.repeat(i, len(df_c.index))
        if 'payment' in i[:20]:
            df_c['value_this_period'] = df_c['value_this_period'].apply(lambda x: -x)
            df_c['value_last_period'] = df_c['value_last_period'].apply(lambda x: -x)
            df_c['value_diff'] = df_c['value_this_period'] - df_c['value_last_period']
        df_ps = pd.concat([df_c,df_ps], ignore_index = True)
    except:
        continue
df_ps['value_this_period'] = df_ps['value_this_period'].apply(lambda x: x/1000000)
df_ps['value_last_period'] = df_ps['value_last_period'].apply(lambda x: x/1000000)
df_ps['value_diff'] = df_ps['value_diff'].apply(lambda x: x/1000000)
df_ps = df_ps.merge(pd.ExcelFile('Mapping.xls').parse('cf'), on = ['us-gaap'], how = 'left')

In [33]:

only_investments = total_cash[total_cash.Cat == 'investment'].groupby(['CIK'])['value'].sum().reset_index()
df_ps_agg = df_ps.groupby(['CIK','reporting_period','this_period','GICS Sector'])['value_this_period'].sum().reset_index()
df_ps_agg = df_ps_agg.sort_values(by = ['CIK','reporting_period','this_period'], ascending = False)\
.drop_duplicates(subset = ['CIK'],keep = 'first')

df_ps_agg = df_ps_agg.merge(only_investments,on = ['CIK'], how = 'left').rename(columns = {'value_this_period':'cash_flow','value':'investment_bal'})
df_ps_agg['change %'] = -df_ps_agg['cash_flow']*100/(df_ps_agg['cash_flow']+df_ps_agg['investment_bal'])
df_ps_agg['investment_bal_last_period'] = df_ps_agg['cash_flow']+df_ps_agg['investment_bal']
df_ps_agg = df_ps_agg.sort_values(by = ['change %'])[df_ps_agg['investment_bal_last_period'] >= 0]
df_ps_agg['valid_cash_flow'] = np.repeat('v', len(df_ps_agg.index))
df_ps_agg.groupby(by = ['reporting_period'])['change %'].describe()
############
#The corporate investments exhibit net outflows of 6~14% of original investment balance regardless of reporting periods of the entities.
############
#df_ps_agg.groupby(by = ['reporting_period']).apply(lambda x: np.average(x['change %'], weights = x['investment_bal'])).reset_index().rename(columns = {0:'weighted_change %'})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,count,mean,std,min,25%,50%,75%,max
reporting_period,,,,,,,,
3,10.0,1.365119e+02,463.432983,-49.670963,-21.519742,-8.547422,10.918498,1.453765e+03
6,133.0,inf,NaN,-79.678166,-23.863636,-0.614754,5.901201,inf
9,24.0,2.044722e+04,100142.148675,-92.398922,-16.955206,-0.869069,6.374812,4.906000e+05
12,16.0,-1.367616e+01,34.781367,-97.314890,-32.566209,-5.558808,5.399347,5.500000e+01


In [31]:
df_ps_agg[df_ps_agg['investment_bal_last_period'] < 0]

,CIK,reporting_period,this_period,GICS Sector,cash_flow,investment_bal,change %,investment_bal_last_period


In [111]:
###################
#Prominent  Consumer Staple companies - Pepsico & ESTEE LAUDER, all reported net investment decrease
#Material: DowDuPont
#Healthcare: Amgen, Anthem, GILEAD
###################
df_ps_agg.groupby(by = ['GICS Sector']).apply(lambda x: np.average(x['change %'], weights = x['investment_bal'])).reset_index().rename(columns = {0:'weighted_change %'})

,GICS Sector,weighted_change %
0,Consumer Discretionary,-7.628350
1,Consumer Staples,-35.268713
2,Energy,4.111967
3,Health Care,-8.482201
4,Industrials,-5.717812
5,Information Technology,-7.363309
6,Materials,-16.657373
7,Utilities,-2.558277


In [35]:
#####################
#Data for Visualization
#Main 是有篩選過last period investment balance 和reporting period有誤的
#####################
df_main = only_investments.rename(columns = {'value':'investment_bal'}).merge(agg, on = ['CIK'], how = 'outer')\
.merge(df_ps_agg, on = ['CIK','investment_bal','GICS Sector'], how = 'left').rename(columns = {'value':'total_cash'})
df_main.loc[df_main['investment_bal_last_period'].isnull(),'investment_bal_last_period'] = df_main.loc[df_main['investment_bal_last_period'].isnull(),'investment_bal']
df_main.to_excel('Main.xls', index = False)

df_ps_agg[['CIK', 'reporting_period', 'this_period', 'GICS Sector']]\
.merge(df_ps,on = ['CIK', 'reporting_period', 'this_period', 'GICS Sector'], how = 'left')\
.to_excel('Cash Flow.xls',index = False)

investment.to_excel('investment_composition.xls', index = False)